In [ ]:
%cd ..

In [ ]:
from model.inpatients import InpatientsModel
from model.outpatients import OutpatientsModel
from model.aae import AaEModel
from model.helpers import load_params
from model.health_status_adjustment import HealthStatusAdjustmentInterpolated
from model.model_iteration import ModelIteration

import os
from tqdm.notebook import tqdm

In [ ]:
params = load_params("queue/sample_params.json")
rp = InpatientsModel.generate_run_params(params)

params["efficiencies"]["ip"] = {
    k: v for k, v in params["efficiencies"]["ip"].items() if not k.startswith("bads")
}

In [ ]:
def check(model):
    def run(ds):
        params["dataset"] = ds
        try:
            hsa = HealthStatusAdjustmentInterpolated(f"data/2019/{ds}/", "2019")
            m = model(params, "data", hsa, rp)
            ModelIteration(m, 0)
            return None
        except Exception as e:
            return e

    r = {}
    for d in tqdm(os.listdir("data/2019")):
        res = run(d)
        if res is not None:
            r[d] = res
    return r

In [ ]:
check(InpatientsModel)

R0B, R0D, RP4: missing files

RP4 is GOSH - missing demographics file

In [ ]:
check(OutpatientsModel)

In [ ]:
check(AaEModel)

# Clean up old datasets

In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import InteractiveBrowserCredential
import os

cred = InteractiveBrowserCredential()
client = DataLakeServiceClient("https://nhpsa.dfs.core.windows.net/", credential=cred)

local_paths = os.listdir("../data/2019")

In [ ]:
fs = client.get_file_system_client("inputs-data")
azure_paths = [
    p["name"][4:]
    for p in fs.get_paths(path="dev", recursive=False)
    if not p["name"].endswith("rds")
]

# dry run:
set(azure_paths) - set(local_paths)
# [ fs.get_directory_client(f"dev/{p}").delete_directory() for p in set(azure_paths) - set(local_paths) ]

In [ ]:
fs = client.get_file_system_client("data")
azure_paths = [
    p["name"][9:]
    for p in fs.get_paths(path="dev/2019", recursive=False)
    if not p["name"].endswith("rds")
]

# dry run:
set(azure_paths) - set(local_paths)
# [ fs.get_directory_client(f"dev/2019/{p}").delete_directory() for p in set(azure_paths) - set(local_paths) ]